# CHSH Game

The **CHSH Game** quantum kata is a series of exercises designed
to get you familiar with the CHSH game.

In it, two players (Alice and Bob) try to win the following game:

Each of them is given a bit (Alice gets X and Bob gets Y), and
they have to return new bits (Alice returns A and Bob returns B)
so that X ∧ Y = A ⊕ B. The trick is, they can not communicate during the game.

```
* '∧' is the standard bitwise AND operator.
* '⊕' is the exclusive or, or XOR operator, so 'P ⊕ Q' is true if exactly one of P and Q is true.
```

* You can read more about CHSH game in the [lecture notes](https://cs.uwaterloo.ca/~watrous/CPSC519/LectureNotes/20.pdf) by
  John Watrous. 
* Q# Samples repository has [an implementation of the CHSH
  game](https://github.com/microsoft/Quantum/tree/master/samples/algorithms/chsh-game)
  that includes an explanation of the history and theory behind the game.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl/⌘+Enter.


To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [1]:
%package Microsoft.Quantum.Katas::0.12.20072031

Adding package Microsoft.Quantum.Katas::0.12.20072031: done!

Microsoft.Quantum.Standard::0.12.20072031, Microsoft.Quantum.Katas::0.12.20072031

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Part I. Classical CHSH


### Task 1.1. Win Condition
**Input:** 

  1. Alice and Bob's starting bits (X and Y),

  2. Alice and Bob's output bits (A and B).

**Output:** 
True if Alice and Bob won the CHSH game, that is, if X ∧ Y = A ⊕ B, and false otherwise.

In [3]:
%kata T11_WinCondition_Test 

open Microsoft.Quantum.Logical;

function WinCondition (x : Bool, y : Bool, a : Bool, b : Bool) : Bool {
    // There are four inputs possible, (0,0), (0,1), (1,0), and (1,1), each with 25% probability.
    // In order to win, Alice and Bob have to output opposite answers if the input is (1,1), and answer the same otherwise.

    let p = And(x, y);
    let u = Xor(a, b);
    return (p == u);
}

Success!

### Task 1.2. Alice and Bob's classical strategy

In this task you have to implement two functions, one for Alice's classical strategy and one for Bob's.
Note that they are covered by one test, so you have to implement both to pass the test. Once you implement one of the strategies, execute its cell - it will fail with the error message indicating that the other strategy is not implemented yet. Once you implement the second strategy, execute its cell to get the test result.

**Input:** Alice's starting bit (X).

**Output:** The bit that Alice should output (A) to maximize their chance of winning.

In [4]:
%kata T12_ClassicalStrategy_Test 

operation AliceClassical (x : Bool) : Bool {
    // If Alice and Bob always return "TRUE", they will post a 75% win rate
    // since TRUE ⊕ TRUE = FALSE, and the AND operation on their input bits will be false with 75% probability.
    return true;
}

Bob's strategy in task 1.2 not implemented yet
Try again!


**Input:** Bob's starting bit (Y).

**Output:** The bit that Bob should output (B) to maximize their chance of winning.

In [5]:
%kata T12_ClassicalStrategy_Test 

operation BobClassical (y : Bool) : Bool {
    // Alternatively, Alice and Bob could agree to always return FALSE to achieve the same 75% win chances.
    return true;
}

Win rate 0.745


Success!

## Part II. Quantum CHSH

In the quantum version of the game, the players still can not
communicate during the game, but they are allowed to share 
qubits from a Bell pair before the start of the game.


### Task 2.1. Entangled pair

**Input:** An array of two qubits in the $|00\rangle$ state.

**Goal:** Create a Bell state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} \big( |00\rangle + |11\rangle \big)$ on these qubits.

In [6]:
%kata T21_CreateEntangledPair_Test 

operation CreateEntangledPair (qs : Qubit[]) : Unit {
    H(qs[0]);
    // This performs a Hadamard transform on the first qubit
    // which produces the intermediate state (|00> + |10>) / sqrt(2).
    
    CX(qs[0], qs[1]);
    // CX (Controlled X, Controlled NOT, CNOT) operates on two qubits, putting the second qubit through a NOT gate
    // if and only if the first qubit is '1'.
    
    // The 4x4 operator matrix for CX is:
    // [1  0  0  0]
    // [0  1  0  0]
    // [0  0  0  1]
    // [0  0  1  0]
    
    // The original state |00> corresponds to the two-qubit amplitude vector [1, 0, 0, 0].
    
    // The intermediate state after the Hadamard transform is given by the column vector [1/sqrt(2), 0, 1/sqrt(2), 0].
    
    // The CX operator changes this vector to [1/sqrt(2), 0, 0, 1/sqrt(2)], which corresponds to the Bell state desired.
}

Success!

### Task 2.2. Alice's quantum strategy

**Inputs:**

  1. Alice's starting bit (X),

  2. Alice's half of Bell pair she shares with Bob.

**Goal:** Measure Alice's qubit in the Z basis if her bit is 0 (false), or the X basis if her bit is 1 (true), and return the result.
The state of the qubit after the operation does not matter.

In [20]:
%kata T22_AliceQuantum_Test 

open Microsoft.Quantum.Measurement;

operation AliceQuantum (bit : Bool, qubit : Qubit) : Bool {
    if (bit) {
        let q = MResetX(qubit);
        return (q == One);
    }
    else {
        let q = MResetZ(qubit);
        return (q == One);
    }
}

Success!

### Task 2.3. Rotate Bob's qubit

**Inputs:**

  1. The direction to rotate: true for clockwise, false for counterclockwise,

  2. Bob's qubit.

**Goal:** Rotate the qubit $\frac{\pi}{8}$ radians around the Y axis in the given direction.

<br/>
<details>
    <summary>Need a hint? Click <a href="https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.ry">here</a></summary>
  Ry operation applies a rotation by a given angle in counterclockwise direction.
</details>

In [17]:
%kata T23_RotateBobQubit_Test 

open Microsoft.Quantum.Math;

operation RotateBobQubit (clockwise : Bool, qubit : Qubit) : Unit {
    mutable dir = PI() / 8.0;
    if (clockwise) {
        set dir = dir * -1.0;
    }                            // In the Ry matrix entries, the trig functions are actually performed on theta/2
    Ry(2.0 * dir, qubit);        // so the angle input into the function has to be doubled.
}

Success!

### Task 2.4. Bob's quantum strategy

**Inputs:**

  1. Bob's starting bit (Y),

  2. Bob's half of Bell pair he shares with Alice.

**Goal:** Measure Bob's qubit in the $\frac{\pi}{8}$ basis if his bit is 0 (false), or the $-\frac{\pi}{8}$ basis if his bit is 1 (true), and return the result.
The state of the qubit after the operation does not matter.

In [25]:
%kata T24_BobQuantum_Test 

open Microsoft.Quantum.Measurement;

operation BobQuantum (bit : Bool, qubit : Qubit) : Bool {
    let antibit = not bit;
    RotateBobQubit(antibit, qubit);     // Measuring a qubit in the (theta) basis is the same as rotating the qubit
    let q = MResetZ(qubit);             // by (theta), clockwise, and then making a standard measurement in the Z basis.
    return (q == One);
}

Success!

### Task 2.5. Play the CHSH game using the quantum strategy

**Input:**
Operations that return Alice and Bob's output bits (A and B) based on their quantum
strategies and given their respective qubits from the Bell pair.
Alice and Bob have already been told what their starting bits X and Y are.

**Goal:** Return Alice and Bob's output bits (A, B).
Note that this task uses strategies `AliceQuantum` and `BobQuantum`, which you've implemented in tasks 2.2 and 2.4, respectively.

In [26]:
%kata T25_PlayQuantumCHSH_Test 

operation PlayQuantumCHSH (askAlice : (Qubit => Bool), askBob : (Qubit => Bool)): (Bool, Bool) {
    using (bell = Qubit[2]) {
        CreateEntangledPair(bell);
        
        let A = askAlice(bell[0]);
        let B = askBob(bell[1]);
        
        return (A, B);
    }
}

Success!

### Discussion: probability of victory for quantum strategy

In the arguments to follow, we make the claim that the above quantum strategy adopted by Alice and Bob delivers a win rate of $\frac{2 + \sqrt{2}}{4}$, or about 85.36%.

First, consider the outcome if Alice and Bob simply measure their qubits in the Z basis without manipulating them at all.  Because of the entanglement inherent to the Bell state they hold, their measurements will always agree (i.e. both true or both false).  This will suffice for victory in the three scenarios (0,0), (0,1) and (1,0) and fail for (1,1), so their win clip is 75%, the same as that for the crude classical strategies of invariably returning both true or both false.

Now incorporate the optimal quantum strategy.  A probability "wavefunction" for two qubits is given by the following length-4 vector:

$
\begin{bmatrix}
\psi_{00}\\
\psi_{01}\\
\psi_{10}\\
\psi_{11}
\end{bmatrix}
$

where $\psi$ represents the "wavefunction" amplitude for the two-qubit combination.  $\psi^2$ gives the probability of observing that combination in the qubit pair upon measurement.  The initial state $|00\rangle$ has $\psi_{00} = 1$ and $\psi_{01} = \psi_{10} = \psi_{11} = 0$.  The Bell state has an amplitude vector as follows:

$
\begin{bmatrix}
1/\sqrt{2}\\
0\\
0\\
1/\sqrt{2}
\end{bmatrix}
$

To circumvent the unwieldy manipulation of nested radicals, we will hereafter reference these vectors and matrices by their decimal approximation:

$
\begin{bmatrix}
0.7071\\
0\\
0\\
0.7071
\end{bmatrix}
$

#### Alice holds bit 0

First consider the cases where Alice's bit is 0, so she simply measures in the Z basis as above.  When Bob's bit is 0, he rotates his qubit clockwise by $\pi/8$, which corresponds to the operator

$
\begin{bmatrix}
0.9239 & 0.3827 & 0 & 0\\
-0.3827 & 0.9239 & 0 & 0\\
0 & 0 & 0.9239 & 0.3827\\
0 & 0 & -0.3827 & 0.9239
\end{bmatrix}
$

This performs the $R_y$ rotation by $\pi/8$ radians clockwise on Bob's qubit while leaving Alice's be.  If Bob's bit were 1, he would rotate his qubit counterclockwise by $\pi/8$, applying the highly similar operator

$
\begin{bmatrix}
0.9239 & -0.3827 & 0 & 0\\
0.3827 & 0.9239 & 0 & 0\\
0 & 0 & 0.9239 & -0.3827\\
0 & 0 & 0.3827 & 0.9239
\end{bmatrix}
$

Therefore, when Alice has 0, the application of the rotation operator to the Bell state gives

$
\begin{bmatrix}
0.6533 & {or} & 0.6533\\
-0.2706 & {or} & 0.2706\\
0.2706 & {or} & -0.2706\\
0.6533 & {or} & 0.6533
\end{bmatrix}
$

depending on whether Bob holds 0 (left-hand case) or 1 (right-hand case).  The result of AND on their bits will always be 0; thus they win when their outputs agree.  These two cases correspond to the top and bottom elements of the vector above, with a combined probability of $(0.6533)^2 + (0.6533)^2 = 0.4268 + 0.4268 = 0.8536$, so they have an 85.36% win chance.

#### Alice holds bit 1

When Alice holds bit 1, she measures in basis X (or, equivalently, Hadamard-transforms her qubit, leaving Bob's be, before making her Z-basis measurement).  This corresponds to applying the operator

$
\begin{bmatrix}
0.7071 & 0 & 0.7071 & 0\\
0 & 0.7071 & 0 & 0.7071\\
0.7071 & 0 & -0.7071 & 0\\
0 & 0.7071 & 0 & -0.7071
\end{bmatrix}
$

to the Bell state, resulting in a vector of:

$
\begin{bmatrix}
0.5\\
0.5\\
0.5\\
-0.5
\end{bmatrix}
$

Now, one of the two rotation operators is applied depending on what bit Bob holds, transforming this vector into:

$
\begin{bmatrix}
0.6533 & {or} & 0.2706\\
0.2706 & {or} & 0.6533\\
0.2706 & {or} & 0.6533\\
-0.6533 & {or} & -0.2706
\end{bmatrix}
$

When Bob holds 0, they still want to return the same parity, which they again do with 85.36% probability (left-hand case).  But when Bob holds 1, the AND condition is now true and the players want to answer in opposite parity.  This corresponds to the second and third elements of the above vector (right-hand case).  Thanks to the "magic" of the combination of the counterclockwise rotation and Hadamard transform, they now do this with probability $(0.6533)^2 + (0.6533)^2 = 0.8536$ and thus 85.36% becomes their win odds once more.

#### Side notes

* If Bob never rotated his qubit, their entangled state would remain the Bell state if Alice held bit 0 and the state corresponding to $(|00\rangle + |01\rangle + |10\rangle - |11\rangle) / 2$ if Alice held bit 1.  While she and Bob would have a lock against the 0 bit, they would be on a coin flip if she held bit 1 and thus their win chance would revert to 75% again.

* It can be proven that Alice and Bob cannot surpass an overall win probability of 85.36% in the CHSH game.  This entails a higher-level discussion of quantum observable theory, for instance see https://en.wikipedia.org/wiki/Tsirelson's_bound.